In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Get:7 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Get:12 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:13 

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-12-12 02:21:28--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  5.77MB/s    in 0.2s    

2021-12-12 02:21:28 (5.77 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Beauty_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+---------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|          review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+---------------------+-----------+
|         US|    1797882|R3I2DHQBR577SS|B001ANOOOE|       2102612|The Naked Bee Vit...|          Beauty|          5|            0|          0|   N|                Y|          Five Stars| Love this, excell...| 2015-08-31|
|         US|   18381298|R1QNE9NQFJC2Y4|B0016J22EQ|     106393691|Alba Botanica Sun...|          Beauty|          5|

In [5]:
from pyspark.sql.functions import to_date
# Read in the Review dataset as a DataFrame
df = df.dropna()
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+---------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|          review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+---------------------+-----------+
|         US|    1797882|R3I2DHQBR577SS|B001ANOOOE|       2102612|The Naked Bee Vit...|          Beauty|          5|            0|          0|   N|                Y|          Five Stars| Love this, excell...| 2015-08-31|
|         US|   18381298|R1QNE9NQFJC2Y4|B0016J22EQ|     106393691|Alba Botanica Sun...|          Beauty|          5|

In [6]:
# Create the vine_table. DataFrame
vine_df = df.select(["review_id","star_rating","helpful_votes","total_votes","vine","verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3I2DHQBR577SS|          5|            0|          0|   N|                Y|
|R1QNE9NQFJC2Y4|          5|            0|          0|   N|                Y|
|R3LIDG2Q4LJBAO|          5|            0|          0|   N|                Y|
|R3KSZHPAEVPEAL|          5|            0|          0|   N|                Y|
| RAI2OIG50KZ43|          5|            0|          0|   N|                Y|
|R1R30FA4RB5P54|          4|            0|          0|   N|                Y|
|R30IJKCGJBGPJH|          5|            0|          0|   N|                Y|
|R18GLJJPVQ1OVH|          5|            0|          0|   N|                Y|
| R8TVYIJXLYJT0|          5|            0|          0|   N|                Y|
|R1CJGF6M3PVHEZ|          1|            0|          2|   N|     

In [7]:
#1. Filter for >20 total votes
filtered_data_df = vine_df.filter("total_votes>=20")
filtered_data_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2QRUE9REK8OUC|          5|           18|         23|   N|                Y|
|R2POXRW6PXHXZG|          5|           25|         30|   N|                Y|
| RZN43XRZ89IIJ|          3|           10|         27|   N|                N|
|R29Q748WSHZ3SN|          5|           23|         24|   N|                Y|
| R46UL5G5HEPRZ|          5|           18|         20|   N|                N|
|R2UQHSNWU6WTZX|          5|           74|         76|   N|                N|
|R2HSRI3D6E2M9Y|          5|           69|         71|   N|                Y|
|R17ARFSDV555EW|          1|           29|         29|   N|                Y|
|R3GL5156FFEDQA|          5|           20|         21|   N|                Y|
|R29KQJC9PIYWQS|          5|           25|         26|   N|     

In [8]:
#2. Filter for >20 total votes & >50% helpful
helpful_df = filtered_data_df.filter("(helpful_votes/total_votes)>=0.5")
helpful_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2QRUE9REK8OUC|          5|           18|         23|   N|                Y|
|R2POXRW6PXHXZG|          5|           25|         30|   N|                Y|
|R29Q748WSHZ3SN|          5|           23|         24|   N|                Y|
| R46UL5G5HEPRZ|          5|           18|         20|   N|                N|
|R2UQHSNWU6WTZX|          5|           74|         76|   N|                N|
|R2HSRI3D6E2M9Y|          5|           69|         71|   N|                Y|
|R17ARFSDV555EW|          1|           29|         29|   N|                Y|
|R3GL5156FFEDQA|          5|           20|         21|   N|                Y|
|R29KQJC9PIYWQS|          5|           25|         26|   N|                Y|
| RYWUJT2A33VJH|          5|           20|         22|   N|     

In [9]:
#3. Filter for paid reviews that are >50% helpful
paid_helpful_df = helpful_df.filter("vine == 'Y'")
paid_helpful_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R106V6GUNWRSSS|          5|          857|        889|   Y|                N|
|R37N8KZS48W36A|          5|           36|         37|   Y|                N|
|R1LNEQ2MQE03PD|          5|           49|         66|   Y|                N|
|R1ECPWPUNK36ES|          5|           29|         31|   Y|                N|
|R16BUM7UQZLOM6|          3|          107|        112|   Y|                N|
| R7NWIOCA5RVPR|          5|           19|         26|   Y|                N|
|R3NTGOCUZB33JK|          5|           40|         47|   Y|                Y|
| RQX3RHGMG7UBN|          5|          186|        200|   Y|                N|
|R3E6QYKZDST042|          4|           33|         39|   Y|                N|
|R2GR106PWUN85O|          4|           36|         39|   Y|     

In [10]:
#4. Filter for unpaid reviews that are >50% helpful
unpaid_helpful_df = helpful_df.filter("vine == 'N'")
unpaid_helpful_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2QRUE9REK8OUC|          5|           18|         23|   N|                Y|
|R2POXRW6PXHXZG|          5|           25|         30|   N|                Y|
|R29Q748WSHZ3SN|          5|           23|         24|   N|                Y|
| R46UL5G5HEPRZ|          5|           18|         20|   N|                N|
|R2UQHSNWU6WTZX|          5|           74|         76|   N|                N|
|R2HSRI3D6E2M9Y|          5|           69|         71|   N|                Y|
|R17ARFSDV555EW|          1|           29|         29|   N|                Y|
|R3GL5156FFEDQA|          5|           20|         21|   N|                Y|
|R29KQJC9PIYWQS|          5|           25|         26|   N|                Y|
| RYWUJT2A33VJH|          5|           20|         22|   N|     

In [25]:
#5a. Total number of reviews (note: where there are >20 votes, and >50% helpful rating)
total_reviews= helpful_df.count()
total_reviews

74702

In [27]:
#5b. Total number of 5-star reviews (note: where there are >20 votes, and >50% helpful rating)
five_stars= helpful_df.filter("star_rating =='5'").count()
five_stars

43408

In [21]:
#5c. The percentage of PAID reviews that have a 5-star Rating (note: these are based on the "helpful dataset")

# Find the Total Number of Paid Reviews
total_paid_reviews= paid_helpful_df.count()

# Find the total number of paid reviews that have a 5-star rating
paid_five_stars= paid_helpful_df.filter("star_rating =='5'").count()

# Find the percentage of paid reviews that have a 5-star rating

paid_percentage= paid_five_stars/total_paid_reviews*100

total_paid_reviews, paid_five_stars, paid_percentage

(647, 229, 35.394126738794434)

In [22]:
#5d. The percentage of UNPAID reviews that have a 5-star Rating (note: these are based on the "helpful dataset")

# Find the Total Number of Unpaid Reviews
total_unpaid_reviews= unpaid_helpful_df.count()

# Find the total number of unpaid reviews that have a 5-star rating
unpaid_five_stars= unpaid_helpful_df.filter("star_rating =='5'").count()

# Find the percentage of unpaid reviews that have a 5-star rating

unpaid_percentage= unpaid_five_stars/total_unpaid_reviews*100

total_unpaid_reviews, unpaid_five_stars, unpaid_percentage

(74055, 43179, 58.30666396597124)